In [ ]:
import requests, pandas as pd

In [ ]:
r = requests.get('https://servicodados.ibge.gov.br/api/v1/localidades/paises?view=nivelado')

In [ ]:
pd.read_json('https://servicodados.ibge.gov.br/api/v1/localidades/estados?view=nivelado', orient='records')

In [ ]:
from ibge.localidades import Estados

Estados().json.

In [ ]:
pd.read_json('https://servicodados.ibge.gov.br/api/v1/localidades/municipios?view=nivelado', orient='records')

In [ ]:
import pandas as pd
import dags.DW_TOOLS as dwt

con = dwt.create_connection(
    server='10.3.152.103',
    database='saude_mental',
    username='postgres',
    password='postgres',
    port=5432)

conn = con.connect()

In [ ]:
query = """
    SELECT DISTINCT
        null as sk_municipio
        , stg."municipio-id" as cd_municipio
        , stg."municipio-nome" as no_municipio
        , stg."microrregiao-id" as cd_microrregiao
        , stg."microrregiao-nome" as no_microrregiao
        , stg."mesorregiao-id" as cd_mesorregiao
        , stg."mesorregiao-nome" as no_mesorregiao
        , stg."regiao-imediata-id" as cd_regiao_imediata
        , stg."regiao-imediata-nome" as no_regiao_imediata
        , stg."regiao-intermediaria-id" as cd_regiao_intermediaria
        , stg."regiao-intermediaria-nome" as no_regiao_intermediaria
        , stg."uf-id" as cd_uf
        , stg."uf-sigla" as ds_sigla_uf
        , stg."uf-nome" as no_uf
        , stg."regiao-id" as cd_regiao
        , stg."regiao-sigla" as ds_sigla_regiao
        , stg."regiao-nome" as no_regiao
    FROM stg.stg_municipios stg
    LEFT JOIN dw.d_municipio dim
        ON (stg."municipio-id" = dim.cd_municipio)
    WHERE dim.cd_municipio IS NULL
"""

df = dwt.read_table_from_sql(query, con)

In [ ]:
df[df['cd_municipio'] == 90]

In [ ]:
query = f"""
    SELECT DISTINCT
        sp."pais-m49" AS cd_naturalidade,
        sp."pais-nome" AS ds_naturalidade,
        'PAÍS' AS ds_nivel
    FROM stg.stg_pais sp
    UNION
    SELECT DISTINCT
        (su."uf-id" + 800) AS cd_naturalidade,
        su."uf-nome" AS ds_naturalidade,
        'UF' AS ds_nivel
    FROM stg.stg_uf su
"""

df = dwt.read_table_from_sql(query, con)
df

In [ ]:
df = df.astype({"cd_naturalidade": "Int64"})
duplicates = df[df['cd_naturalidade'].duplicated()]['cd_naturalidade']

In [ ]:
df = df.query("not cd_naturalidade.duplicated() or ds_nivel != 'PAÍS'")
df

In [ ]:
df = pd.read_sql_table('d_municipio', con.connect(), 'dw', columns=['sk_municipio', 'cd_municipio'])

In [11]:
import pandas as pd
import datetime as dt
import dags.DW_TOOLS as dwt
import sqlalchemy as sa

schema = 'dw'
tb_name = 'f_obito'


con = dwt.create_connection(
    server='10.3.152.103',
    database='saude_mental',
    username='postgres',
    password='postgres',
    port=5432)

conn = con.connect()

query = f"""
    SELECT DISTINCT
        NULLIF(TRIM(sim.codmunnatu), '')::INTEGER AS cd_municipio_naturalidade,
        NULLIF(TRIM(sim.codmunocor), '')::INTEGER AS cd_municipio_ocorrencia,
        NULLIF(TRIM(sim.codmunres), '')::INTEGER AS cd_municipio_residencia,
        NULLIF(TRIM(sim.ocup), '')::INTEGER AS cd_ocupacao,
        NULLIF(TRIM(sim."natural"), '')::INTEGER AS cd_naturalidade,
        NULLIF(TRIM(sim.racacor), '')::INTEGER AS cd_raca_cor,
        NULLIF(TRIM(sim.causabas), '') AS cd_cid,
        NULLIF(TRIM(sim.esc), '')::INTEGER AS cd_escolaridade,
        NULLIF(TRIM(sim.seriescfal), '')::INTEGER AS nu_serie,
        NULLIF(TRIM(sim.sexo), '')::INTEGER AS cd_sexo,
        NULLIF(TRIM(sim.contador), '')::INTEGER AS cd_registro,
        CASE NULLIF(TRIM(sim.assistmed), '')
            WHEN '1' THEN 'Com assistência'
            WHEN '2' THEN 'Sem assistência'
            WHEN '9' THEN 'Igorado'
            ELSE NULL
        END AS ds_assistencia_medica,
        CASE NULLIF(TRIM(sim.atestante), '')
            WHEN '1' THEN 'Sim'
            WHEN '2' THEN 'Substituto'
            WHEN '3' THEN 'IML'
            WHEN '4' THEN 'SVO'
            WHEN '5' THEN 'Outros'
            ELSE NULL
        END AS ds_atestante,
        CASE NULLIF(TRIM(sim.circobito), '')
            WHEN '1' THEN 'Acidente'
            WHEN '2' THEN 'Suicídio'
            WHEN '3' THEN 'Homicídio'
            WHEN '4' THEN 'Outros'
            WHEN '9' THEN 'Ignorado'
            ELSE NULL
        END AS ds_circunstancia_obito,
        REGEXP_REPLACE(NULLIF(TRIM(sim.crm), ''), '[^0-9]*', '', 'g') AS nu_crm_medico_atestante,
        TO_DATE(NULLIF(TRIM(sim.dtcadastro), ''), 'DDMMYY') AS dt_cadastro,
        TO_DATE(NULLIF(TRIM(sim.dtatestado), ''), 'DDMMYY') AS dt_atestado,
        TO_DATE(NULLIF(TRIM(sim.dtobito), ''), 'DDMMYY') AS dt_obito,
        TO_DATE(NULLIF(TRIM(sim.dtnasc), ''), 'DDMMYY') AS dt_nascimento_falecido,
        CASE
            WHEN NULLIF(TRIM(sim.idade), '') = '000'
                THEN -2
            WHEN LEFT(NULLIF(TRIM(sim.idade), ''),1) IN ('0','1','2','3','4')
                THEN RIGHT(NULLIF(TRIM(sim.idade), ''),2)::INTEGER
            WHEN LEFT(NULLIF(TRIM(sim.idade), ''),1) = '5'
                THEN (100 + RIGHT(NULLIF(TRIM(sim.idade), ''),2)::INTEGER)
            ELSE -2
        END AS vl_idade_falecido,
        CASE
            WHEN NULLIF(TRIM(sim.idade), '') = '000' THEN 'Não Informado'
            ELSE
                CASE LEFT(NULLIF(TRIM(sim.idade), ''),1)
                    WHEN '0' THEN 'MINUTOS'
                    WHEN '1' THEN 'HORAS'
                    WHEN '2' THEN 'DIAS'
                    WHEN '3' THEN 'MESES'
                    WHEN '4' THEN 'ANOS'
                    WHEN '5' THEN 'ANOS'
                    ELSE 'Não Informado'
                END
        END AS ds_unidade_idade_falecido,
        NOW() as dt_carga
    FROM stg.stg_sim sim
    -- LEFT JOIN {schema}.{tb_name} fato
    --    ON (NULLIF(TRIM(sim.contador), '')::INTEGER = fato.cd_registro)
    -- WHERE fato.cd_registro IS NULL
"""

with con.connect() as conn:
    result = conn.execute(sa.text(query))

In [12]:
result.columns

<bound method Result.columns of <sqlalchemy.engine.cursor.CursorResult object at 0x0000013587A75400>>

In [ ]:
col_names = {
    'contador': 'cd_registro',
    'crm': 'nu_crm_medico_atestante',
    'dtcadastro': 'dt_cadastro',
    'dtatestado': 'dt_atestado',
    'dtobito': 'dt_obito',
    'dtnasc': 'dt_nascimento_falecido',
    'sk_cid': 'sk_cid_causa_obito',
    'sk_escolaridade': 'sk_escolaridade_falecido',
    'sk_municipio': 'sk_municipio_ocorrencia_obito', 
    'sk_ocupacao': 'sk_ocupacao_falecido'
}

dtypes = {
    'sk_municipio_naturalidade_falecido': 'Int64',
    'sk_municipio_ocorrencia_obito': 'Int64',
    'sk_municipio_residencia_falecido': 'Int64',
    'sk_ocupacao_falecido': 'Int64',
    'sk_naturalidade': 'Int64',
    'sk_raca_cor': 'Int64',
    'sk_cid_causa_obito': 'Int64',
    'sk_escolaridade_falecido': 'Int64',
    'sk_sexo': 'Int64',
    'cd_registro': 'Int64',
    'ds_assistencia_medica': 'string',
    'ds_atestante': 'string',
    'ds_circunstancia_obito': 'string',
    'nu_crm_medico_atestante': 'Int64',
    'dt_cadastro': 'datetime64[ns]',
    'dt_atestado': 'datetime64[ns]',
    'dt_obito': 'datetime64[ns]',
    'dt_hora_obito': 'string',
    'dt_nascimento_falecido': 'datetime64[ns]',
    'vl_idade_falecido': 'int',
    'ds_unidade_idade_falecido': 'string',
    'dt_carga': 'datetime64[ns]'
} 

fillna = {
    'sk_municipio_naturalidade_falecido': -2,
    'sk_municipio_ocorrencia_obito': -2,
    'sk_municipio_residencia_falecido': -2,
    'sk_ocupacao_falecido': -2,
    'sk_naturalidade': -2,
    'sk_raca_cor': -2,
    'sk_cid_causa_obito': -2,
    'sk_escolaridade_falecido': -2,
    'sk_sexo': -2,
    'cd_registro': -2,
    'ds_assistencia_medica': 'Não informado',
    'ds_atestante': 'Não informado',
    'ds_circunstancia_obito': 'Não informado',
    'nu_crm_medico_atestante': -2,
    'dt_cadastro': pd.to_datetime('01/01/1900'),
    'dt_atestado': pd.to_datetime('01/01/1900'),
    'dt_obito': pd.to_datetime('01/01/1900'),
    'dt_hora_obito': '0000',
    'dt_nascimento_falecido': pd.to_datetime('01/01/1900'),
    'vl_idade_falecido': -2,
    'ds_unidade_idade_falecido': 'Não informado'
}

In [ ]:
tbl_fato = tbl_fato.rename(columns=col_names).filter(dtypes.keys())

tbl_fato['dt_cadastro'] = pd.to_datetime(tbl_fato['dt_cadastro'], format='%d%m%Y')
tbl_fato['dt_atestado'] = pd.to_datetime(tbl_fato['dt_atestado'], format='%d%m%Y')
tbl_fato['dt_obito'] = pd.to_datetime(tbl_fato['dt_obito'], format='%d%m%Y')
tbl_fato['dt_nascimento_falecido'] = pd.to_datetime(tbl_fato['dt_nascimento_falecido'], format='%d%m%Y')

In [ ]:
tbl_fato = tbl_fato.astype(dtypes).fillna(fillna)

In [ ]:
tbl_fato[tbl_fato['dt_obito'] == pd.to_datetime('2011-07-09')]

In [ ]:
mun = pd.read_excel('ftp://ftp.datasus.gov.br/dissemin/publicos/SIM/CID10/TABELAS/CADMUN.xls')

In [ ]:
mun